In [29]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os
from sacrebleu import corpus_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
words_to_load = 50000
PAD_IDX = 0
UNK_IDX = 1
SOS_IDX = 2
EOS_IDX = 3

with open('cc.zh.300.vec') as f:
    loaded_embeddings_ft = np.zeros((words_to_load+3, 300))
    words_ft = {}
    idx2words_ft = {}
    ordered_words_ft = []
    ordered_words_ft.extend(['<pad>', '<unk>', '<s>'])
    loaded_embeddings_ft[0,:] = np.zeros(300)
    loaded_embeddings_ft[1,:] = np.random.normal(size = 300)
    loaded_embeddings_ft[2,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft[i+3, :] = np.asarray(s[1:])
        words_ft[s[0]] = i+3
        idx2words_ft[i+3] = s[0]
        ordered_words_ft.append(s[0])
    words_ft['<pad>'] = PAD_IDX
    words_ft['<unk>'] = UNK_IDX
    words_ft['<s>'] = SOS_IDX
    idx2words_ft[PAD_IDX] = '<pad>'
    idx2words_ft[UNK_IDX] = '<unk>'
    idx2words_ft[SOS_IDX] = '<s>'

In [3]:
#English embedding
with open('wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft_en = np.zeros((words_to_load+4, 300))
    words_ft_en = {}
    idx2words_ft_en = {}
    ordered_words_ft_en = []
    ordered_words_ft_en.extend(['<pad>', '<unk>', '<s>', '</s>'])
    loaded_embeddings_ft_en[0,:] = np.zeros(300)
    loaded_embeddings_ft_en[1,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[2,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[3,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft_en[i+4, :] = np.asarray(s[1:])
        words_ft_en[s[0]] = i+4
        idx2words_ft_en[i+4] = s[0]
        ordered_words_ft_en.append(s[0])
    words_ft_en['<pad>'] = PAD_IDX
    words_ft_en['<unk>'] = UNK_IDX
    words_ft_en['<s>'] = SOS_IDX
    words_ft_en['</s>'] = EOS_IDX
    idx2words_ft_en[PAD_IDX] = '<pad>'
    idx2words_ft_en[UNK_IDX] = '<unk>'
    idx2words_ft_en[SOS_IDX] = '<s>'
    idx2words_ft_en[EOS_IDX] = '</s>'

In [4]:
#read in chinese-english pairs
lines_zh = open('iwslt-zh-en/train.tok.zh',encoding = 'utf-8').read().strip().split('\n')
lines_en = open('iwslt-zh-en/train.tok.en',encoding = 'utf-8').read().strip().split('\n')
lines_zh_test = open('iwslt-zh-en/test.tok.zh',encoding = 'utf-8').read().strip().split('\n')
lines_en_test = open('iwslt-zh-en/test.tok.en',encoding = 'utf-8').read().strip().split('\n')

In [5]:
#add sos and eos in each sentence
def add_sos_eos(lines):
    
    train = []
    for l in lines:
        l = '<s> ' + l + '</s>'
        train.append(l)
    return train
zh_train = add_sos_eos(lines_zh)    
en_train = add_sos_eos(lines_en)
zh_test = add_sos_eos(lines_zh_test)
en_test = add_sos_eos(lines_en_test)

In [6]:
# convert token to id in the dataset
def token2index_dataset(tokens_data,eng = False):
    indices_data = []
    for tokens in tokens_data:
        index_list = []
        for token in tokens.split():
            if eng == False:
                try:
                    index_list.append(words_ft[token])
                except KeyError:
                    index_list.append(UNK_IDX)
            else:
                try:
                    index_list.append(words_ft_en[token])
                except KeyError:
                    index_list.append(UNK_IDX)
        indices_data.append(index_list)
    return indices_data

In [7]:
zh_train_indices = token2index_dataset(zh_train)
en_train_indices = token2index_dataset(en_train,eng = True)
zh_test_indices = token2index_dataset(zh_test)
en_test_indices = token2index_dataset(en_test,eng = True)

In [20]:
#max_sentence_length
length_of_en = [len(x.split()) for x in en_train]
max_sentence_length_en = sorted(length_of_en)[-int(len(length_of_en)*0.01)]
length_of_zh = [len(x.split()) for x in zh_train]
max_sentence_length_zh = sorted(length_of_zh)[-int(len(length_of_zh)*0.01)]
max_sentence_length_en = 50
max_sentence_length_zh = 50

In [21]:
#Create Data Loader
import torch
from torch.utils.data import Dataset

class load_dataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list_s1,data_list_s2):
        """
        @param data_list_zh: list of Chinese tokens 
        @param data_list_en: list of English tokens as TARGETS
        """
        self.data_list_s1 = data_list_s1
        self.data_list_s2 = data_list_s2
        
        assert (len(self.data_list_s1) == len(self.data_list_s2))

    def __len__(self):
        return len(self.data_list_s1)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx_s1 = self.data_list_s1[key][:max_sentence_length_zh]
        token_idx_s2 = self.data_list_s2[key][:max_sentence_length_en]
        return [token_idx_s1, token_idx_s2, len(token_idx_s1), len(token_idx_s2)]

def collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list_s1 = []
    data_list_s2 = []
    length_list_s1 = []
    length_list_s2 = []
    for datum in batch:
        length_list_s1.append(datum[2])
        length_list_s2.append(datum[3])
        padded_vec_zh = np.pad(np.array(datum[0]), 
                                pad_width=((0,max_sentence_length_zh-datum[2])), 
                                mode="constant", constant_values=0)
        padded_vec_en = np.pad(np.array(datum[1]), 
                                pad_width=((0,max_sentence_length_en-datum[3])), 
                                mode="constant", constant_values=0)
        data_list_s1.append(padded_vec_zh[:max_sentence_length_zh])
        data_list_s2.append(padded_vec_en[:max_sentence_length_en])
    #print(type(data_list_s1[0]))
    if torch.cuda.is_available and torch.has_cudnn:
        return [torch.from_numpy(np.array(data_list_s1)).cuda(), torch.from_numpy(np.array(data_list_s2)).cuda(),
                torch.LongTensor(length_list_s1).cuda(), torch.LongTensor(length_list_s2).cuda()]
    else:    
        return [torch.from_numpy(np.array(data_list_s1)), torch.from_numpy(np.array(data_list_s2)),
                torch.LongTensor(length_list_s1), torch.LongTensor(length_list_s2)]
    


In [22]:
BATCH_SIZE = 32
EMBEDDING_SIZE = 300 # fixed as from the input embedding data


train_dataset = load_dataset(zh_train_indices, en_train_indices)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=collate_func,
                                           shuffle=True)

val_dataset = load_dataset(zh_test_indices, en_test_indices)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=collate_func,
                                           shuffle=False)

### Without Attention

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_dim, hidden_size, embed= torch.from_numpy(loaded_embeddings_ft).float(),num_layers=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.emb_dim = emb_dim
        self.num_layers = num_layers 

        self.embedding = nn.Embedding.from_pretrained(embed, freeze=True)
        self.gru = nn.GRU(emb_dim, hidden_size,num_layers=num_layers,batch_first=True)

    def forward(self, data, hidden):
        
        batch_size, seq_len = data.size()
        
        embed = self.embedding(data)
        
        output, hidden = self.gru(embed,hidden)
        #hidden = [n layers * n directions =1 , batch_size, hidden_size ]
        
        return output, hidden

    # initialize the hidden with random numbers
    def initHidden(self,batch_size):
        return torch.randn(self.num_layers, batch_size, self.hidden_size,device=device)

In [17]:
class DecoderRNN(nn.Module):
    def __init__(self,emb_dim,hidden_size, output_size, embed= torch.from_numpy(loaded_embeddings_ft_en).float(),num_layers=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers 
        self.output_size = output_size

        #self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding = nn.Embedding.from_pretrained(embed, freeze=True)
        self.gru = nn.GRU(emb_dim, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, data, hidden):
        embed = self.embedding(data)

        #print("decoder",embed.size())
        
        output = F.relu(embed)
        #print("decoder",output.size())
        
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        
        return output, hidden

    def initHidden(self,batch_size):
        return torch.randn(self.num_layers, batch_size, self.hidden_size,device=device)

In [12]:
teacher_forcing_ratio = 0.5
#input_tensor: list of sentence tensor
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer,
          criterion):
    
    batch_size_1, input_length = input_tensor.size()
    batch_size_2, target_length = target_tensor.size()
    
    
    encoder_hidden = encoder.initHidden(batch_size_1)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()


    #print(target_tensor.size())

    loss = 0

    # encoder_hidden: 1 * batch * hidden size# 
    _, encoder_hidden = encoder(input_tensor, encoder_hidden)
    #print("ssss",encoder_hidden.size())

    decoder_input = torch.tensor(np.array([[SOS_IDX]]*batch_size_1).reshape(1,batch_size_1),device=device)
    decoder_hidden = encoder_hidden


    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    #print(use_teacher_forcing)
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            
            # decoder_output: batchsize * 50003#
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            
            loss += criterion(decoder_output, target_tensor[:,di])
            #print('ssssaaa',loss)

            decoder_input = target_tensor[:,di].unsqueeze(0)  # Teacher forcing
            
            #print(decoder_input.size())
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            #print(decoder_input.size())
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            
            decoder_input = decoder_input.unsqueeze(0)
            #print(decoder_input.size())
            
            loss += criterion(decoder_output, target_tensor[:,di])
            #print(loss)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [28]:
def trainIters(encoder, decoder, n_iters, print_every=1, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    #--------------------------------------------	
    #	
    #    LOAD MODELS	
    #	
    #--------------------------------------------	
    folder = './no_attentation_model'	
    if not os.path.exists(folder):	
        os.makedirs(folder)	

    if os.path.exists('./no_attentation_model/encoder_no_attn.pth'):	
        print('---------------------------------------------------------------------')	
        print('----------------Readind trained model---------------------------------')	
        print('---------------------------------------------------------------------')	
        	
        #read trained models	
        encoder.load_state_dict(torch.load(folder+"/encoder_no_attn.pth"))
        decoder.load_state_dict(torch.load(folder+"/decoder_no_attn.pth"))	
    

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for iter in range(1, n_iters + 1):
        for i, (data_s1, data_s2, lengths_s1, lengths_s2) in enumerate(train_loader):
            input_tensor = data_s1
            target_tensor = data_s2
            #print("train",target_tensor.size())
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if i % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if i % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
                
        # Save the model for every epoch
        print('---------------------------------------------------------------------')	
        print('----------------Saving trained model---------------------------------')	
        print('---------------------------------------------------------------------')	
      
        torch.save(encoder.state_dict(),folder +"/encoder_no_attn.pth")
        torch.save(decoder.state_dict(),folder +"/decoder_no_attn.pth")
    
    return plot_losses




In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [49]:
hidden_size = 300
encoder1 = EncoderRNN(EMBEDDING_SIZE,hidden_size).to(device)
decoder1 = DecoderRNN(EMBEDDING_SIZE,hidden_size, len(ordered_words_ft)).to(device)

##UNCOMMENT TO TRAIN THE MODEL
trainIters(encoder1, decoder1, 5, print_every=50)
#encoder1.load_state_dict(torch.load("encoder.pt"))
#decoder1.load_state_dict(torch.load("decoder.pt"))

0m 0s (- 0m 1s) (1 20%) 0.2160
0m 9s (- 0m 38s) (1 20%) 4.5759
0m 18s (- 1m 15s) (1 20%) 2.8756
0m 28s (- 1m 52s) (1 20%) 2.7281
0m 37s (- 2m 29s) (1 20%) 2.6842
0m 46s (- 3m 6s) (1 20%) 2.6088
0m 55s (- 3m 42s) (1 20%) 2.5778
1m 4s (- 4m 19s) (1 20%) 2.5326
1m 14s (- 4m 56s) (1 20%) 2.6198
1m 23s (- 5m 33s) (1 20%) 2.5535
1m 32s (- 6m 10s) (1 20%) 2.5528
1m 41s (- 6m 46s) (1 20%) 2.5527
1m 50s (- 7m 23s) (1 20%) 2.5319
2m 0s (- 8m 0s) (1 20%) 2.5540
2m 9s (- 8m 37s) (1 20%) 2.4868
2m 18s (- 9m 14s) (1 20%) 2.4715
2m 27s (- 9m 50s) (1 20%) 2.4803
2m 36s (- 10m 27s) (1 20%) 2.4477
2m 45s (- 11m 3s) (1 20%) 2.4318
2m 55s (- 11m 40s) (1 20%) 2.4317
3m 4s (- 12m 18s) (1 20%) 2.5702
3m 13s (- 12m 54s) (1 20%) 2.4071
3m 22s (- 13m 31s) (1 20%) 2.4527
3m 32s (- 14m 8s) (1 20%) 2.4010
3m 41s (- 14m 45s) (1 20%) 2.4010
3m 50s (- 15m 22s) (1 20%) 2.5058
3m 59s (- 15m 58s) (1 20%) 2.4512
4m 8s (- 16m 35s) (1 20%) 2.4622
4m 18s (- 17m 12s) (1 20%) 2.4325
4m 27s (- 17m 49s) (1 20%) 2.3672
4m 36s (-

35m 41s (- 53m 31s) (2 40%) 2.1237
35m 50s (- 53m 45s) (2 40%) 2.0654
35m 59s (- 53m 59s) (2 40%) 2.1104
36m 8s (- 54m 13s) (2 40%) 2.0801
36m 18s (- 54m 27s) (2 40%) 2.0164
36m 27s (- 54m 40s) (2 40%) 1.9929
36m 36s (- 54m 54s) (2 40%) 2.0146
36m 45s (- 55m 8s) (2 40%) 2.0740
36m 54s (- 55m 22s) (2 40%) 2.0445
37m 4s (- 55m 36s) (2 40%) 2.0112
37m 13s (- 55m 50s) (2 40%) 2.1169
37m 22s (- 56m 3s) (2 40%) 2.0249
37m 31s (- 56m 17s) (2 40%) 2.0245
37m 40s (- 56m 31s) (2 40%) 2.0842
37m 50s (- 56m 45s) (2 40%) 2.0639
37m 59s (- 56m 59s) (2 40%) 2.0341
38m 8s (- 57m 12s) (2 40%) 2.0834
38m 18s (- 57m 27s) (2 40%) 2.1803
38m 27s (- 57m 40s) (2 40%) 2.0539
38m 36s (- 57m 54s) (2 40%) 2.0528
38m 45s (- 58m 8s) (2 40%) 2.1115
38m 54s (- 58m 22s) (2 40%) 1.9882
39m 4s (- 58m 36s) (2 40%) 1.9983
39m 13s (- 58m 50s) (2 40%) 2.1225
39m 22s (- 59m 4s) (2 40%) 2.0763
39m 31s (- 59m 17s) (2 40%) 2.0605
39m 41s (- 59m 31s) (2 40%) 2.0331
39m 50s (- 59m 45s) (2 40%) 1.9620
39m 59s (- 59m 59s) (2 40%) 

70m 1s (- 17m 30s) (4 80%) 1.8848
70m 10s (- 17m 32s) (4 80%) 1.9662
70m 19s (- 17m 34s) (4 80%) 1.9272
70m 29s (- 17m 37s) (4 80%) 1.8629
70m 38s (- 17m 39s) (4 80%) 1.9214
70m 47s (- 17m 41s) (4 80%) 1.9410
70m 56s (- 17m 44s) (4 80%) 1.8487
71m 5s (- 17m 46s) (4 80%) 1.8956
71m 14s (- 17m 48s) (4 80%) 1.8537
71m 24s (- 17m 51s) (4 80%) 1.9311
71m 33s (- 17m 53s) (4 80%) 2.0017
71m 42s (- 17m 55s) (4 80%) 1.9379
71m 51s (- 17m 57s) (4 80%) 1.9905
72m 0s (- 18m 0s) (4 80%) 1.9312
72m 10s (- 18m 2s) (4 80%) 1.9308
72m 19s (- 18m 4s) (4 80%) 1.9561
72m 28s (- 18m 7s) (4 80%) 1.9395
72m 37s (- 18m 9s) (4 80%) 1.9053
72m 46s (- 18m 11s) (4 80%) 1.8167
72m 55s (- 18m 13s) (4 80%) 1.9969
73m 5s (- 18m 16s) (4 80%) 1.8963
73m 14s (- 18m 18s) (4 80%) 1.9703
73m 23s (- 18m 20s) (4 80%) 1.8924
73m 32s (- 18m 23s) (4 80%) 1.9120
73m 42s (- 18m 25s) (4 80%) 1.8598
73m 51s (- 18m 27s) (4 80%) 1.9510
74m 0s (- 18m 30s) (4 80%) 2.0242
74m 9s (- 18m 32s) (4 80%) 1.8985


KeyboardInterrupt: 

### evaluate

In [32]:
def index2token_sentence(sentence_batch):
    return ' '.join(idx2words_ft_en[sent.item()] for sent in sentence_batch if sent.item()!=PAD_IDX)

In [47]:
#loader can be test_loader or val_loader
def evaluate(loader, encoder, decoder, beam = False, beam_k = 1):
    bleu_score_list = []
    with torch.no_grad():
        for i, (data_s1, data_s2, lengths_s1, lengths_s2) in enumerate(loader):
            input_tensor = data_s1
            input_length = input_tensor.size()[0]
            #sentence_length to the output length
            sentence_length = data_s2.size()[1]
            encoder_hidden = encoder.initHidden(input_length)

            encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)
            
            #decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
            decoder_input = torch.tensor(np.array([[SOS_IDX]]*input_length).reshape(1,input_length),device=device)

            decoder_hidden = encoder_hidden

#             decoder_attentions = torch.zeros(sentence_length, sentence_length)
            decoded_words_eval = []
#             sequences = [[list(), 1.0]]*input_length
            for di in range(sentence_length):
                decoded_words_sub = []
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                # decoder_attentions[di] = decoder_attention.data
                # topk(1) - softmax probability maximum
                if beam == True:
                    pass
#                     topv, topi = decoder_output.data.topk(beam_k)
#                     #batch loop
#                     C = []
#                     for idx, ind in enumerate(topi):
#                         H, _ = sequences[idx]
#                         for ele in ind:
#                             if ele.item() == EOS_IDX:
#                                 H.append('<EOS>')
#                             else:
#                                 H.append(idx2words_ft_en[ele.item()])
                         
                else:
                    topv, topi = decoder_output.data.topk(1) 
                
                eos_flag = False
                for ind in topi:
                    if eos_flag == False:
                        if ind.item() == EOS_IDX:
                            decoded_words_sub.append('</s>')
                            eos_flag = True
                            #break
                        else:
                            decoded_words_sub.append(idx2words_ft_en[ind.item()])
                    else:
                        decoded_words_sub.append("<pad>")
                        
                
                decoded_words_eval.append(decoded_words_sub)
                
                #swap dimensions of decoded_words to [batch_size * 377]
                
                #decoded_words_new = [[i for i in ele] for ele in list(zip(*decoded_words_eval))]

                #change the dimension
                decoder_input = topi.squeeze().detach()
                decoder_input = decoder_input.unsqueeze(0)
            
            
            pred_num = 0
            listed_predictions = []
            
            
            decoded_words_new = [[i for i in ele] for ele in list(zip(*decoded_words_eval))]
            for token_list in decoded_words_new:
                sent = ' '.join(str(token) for token in token_list if token!="<pad>")
#                 print (sent)
                listed_predictions.append(sent)
                pred_num += 1
                
            ref_num = 0
            listed_reference = []
            for ele in data_s2:
                sent = index2token_sentence(ele)
                #print (tokens)
                #sent = ' '.join(tokens)
#                 print (sent)
                listed_reference.append(sent)
                ref_num += 1
            
            
            bleu_score = corpus_bleu(listed_predictions,[listed_reference])
            
#             uncommon to print prediction and reference
            for i in range(10):
                print ('predicted: ', listed_predictions[i])
                print ('real: ',listed_reference[i])
                print('\n')
            
            
            print('BLEU Score is %s' % (str(bleu_score.score)))
           
        bleu_score_list.append(bleu_score)
        return bleu_score_list, decoded_words_new

In [48]:
bleu_scores, decoded_words_new = evaluate(train_loader,
    encoder1, decoder1)

predicted:  <s> And <unk> <unk> <unk> <unk> <unk>
real:  <s> I <unk> been on 19 hardcore expeditions <unk>


predicted:  <s> And <unk> , , , , , , , , , , , , , , <unk> , <unk>
real:  <s> I mean , if you had to solve one of these problems , Chris , which would you solve ? Terrorism or poverty <unk>


predicted:  <s> And <unk> , <unk> , <unk> , <unk> <unk> <unk> <unk>
real:  <s> I tend to win those arguments , because I know the tricks <unk>


predicted:  <s> And <unk> <unk> <unk>
real:  <s> There we go <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> Elizabeth Lindsey : <unk> humanity <unk> <unk>


predicted:  <s> And <unk> <unk> <unk> <unk>
real:  <s> And I just love this guy <unk>


predicted:  <s> And <unk> , , , , , , , , , , <unk> , <unk> , <unk>
real:  <s> Anyway , this is a song about : we have all this , but what are we going to do with it <unk>


predicted:  <s> And <unk> , , , , , , , , <unk> , <unk> <unk>
real:  <s> You can see the <unk> Treating Fi

predicted:  <s> And <unk> , , , , , , , , , , <unk> , <unk> <unk>
real:  <s> My mother , a clinical <unk> , would occasionally see patients at home in the evening <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> So , what exactly does Global Witness do <unk>


predicted:  <s> And <unk> , , , , , , , , , , , the the the the the the the the the the the the <unk>
real:  <s> The old festivals , they continued to thrive , but from Brighton to Rio to Perth , something new was emerging , and these festivals were really different <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> They are really going to come to operate in this realm <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk>
real:  <s> So imagine -- let <unk> suppose we <unk> doing it <unk>


predicted:  <s> And , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,
real:  <s> So this is the preparation that one of my former <

predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> In Ethiopian crosses , you see this wonderful unfolding of the shape <unk>


predicted:  <s> And <unk> <unk> <unk> <unk>
real:  <s> Has the human mind changed <unk>


predicted:  <s> And , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , <unk>
real:  <s> This is a nonprofit consortium called the Walk Again Project that is putting together scientists from Europe , from here in the United States , and in Brazil together to work to actually get this new body built -- a body that we believe , through the same plastic


predicted:  <s> And <unk> , the the the the the the the the the the the the the the <unk>
real:  <s> Lucas again , in 1876 , showed us - 1 , 21 digits long , was not prime <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> How do we feed a growing world without destroying the planet <unk>


predicted: 

predicted:  <s> And <unk> , the the the the the the the the the the the <unk>
real:  <s> The pattern of Main Street USA -- in fact , this pattern of building downtown blocks , all over the world , is fairly universal <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> So can we explain those 20 numbers <unk>


predicted:  <s> And <unk> <unk> <unk> <unk>
real:  <s> Good morning everybody <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> And that is all of these different things -- the love for my child <unk>


predicted:  <s> And <unk> , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , <unk>
real:  <s> All right , so the guy said electronics . I don <unk> remember anything else . In fact , I don <unk> remember anything that my sixth grade teacher said all year , but I remember electronics <unk>


predicted:  <s> And <unk> , , , , , , , , , , , , , , , , , , , , <unk> , <unk>
real:  <s> I <unk> going to talk a lit

predicted:  <s> And <unk> <unk> <unk>
real:  <s> It <unk> that simple <unk>


predicted:  <s> And <unk> , , , , , , , <unk> , <unk> , <unk>
real:  <s> That came from a 1960 paper on space travel , because , if you think about it , space is pretty awkward <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk>
real:  <s> Let <unk> find a way to join the dance <unk>


predicted:  <s> And <unk> , the the the the the the the the the the the the the the the the the the <unk>
real:  <s> It <unk> as if we <unk> all got 125 light bulbs on all the time , 125 <unk> per day per person is the energy consumption of the <unk>


predicted:  <s> And <unk> , the the the the the the the the the <unk>
real:  <s> And shortly before his 30th birthday in 1958 , <unk> <unk> killed himself <unk>


predicted:  <s> And <unk> , , , , , , , , , , , , , , , , , , , , , , , , <unk>
real:  <s> He was one of the members . The question was : How do you go from a <unk> code to the <unk> code of proteins <unk>


predi

...

predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> Dean <unk> : Your genes are not your <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> But the interesting thing is , you can broaden the search a little bit <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk>
real:  <s> Now , let <unk> see <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> So here <unk> , for example , a picture of sand from Maui <unk>


predicted:  <s> And <unk> <unk> <unk>
real:  <s> Thank you very much <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> It <unk> like a big TED , right <unk>


predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> So where does this trend stop <unk>


predicted:  <s> And <unk> , , , , , , , , , , , <unk> , <unk>
real:  <s> Being a child , and sort of crawling around the house , I remember these Turkish carpets , and there were thes

predicted:  <s> And <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> That <unk> how you think about security -- in terms of the trade-off <unk>


predicted:  <s> And <unk> , <unk> , <unk> , <unk> <unk> <unk> <unk>
real:  <s> Zach and I have always been fascinated observing unexpected things like this <unk>


predicted:  <s> And <unk> , the the the the the the the the the the the the the <unk>
real:  <s> What are the challenges in getting value for money as well as value for many <unk>


predicted:  <s> And <unk> <unk> , <unk> <unk> <unk> <unk> <unk> <unk>
real:  <s> We <unk> other experiences -- we take one item and transfer it to another <unk>


predicted:  <s> And <unk> <unk> , <unk> , <unk> <unk> <unk> <unk>
real:  <s> And now , 14 of the disorders are treatable , two of them actually <unk> <unk>


predicted:  <s> And <unk> , , , , , , , , , , , , <unk> , <unk>
real:  <s> The team from Rice University started to engineer the substance in red wine that makes red wine 

predicted:  <s> And <unk> , , , , , , , , , , , , , , , , , , , , , , , , , , <unk>
real:  <s> We see that with the phenomenon of raising backyard hens and chickens and ducks . And when you work out the math , they say it doesn <unk> work , but the principle is there that it <unk> about being sustainable and taking care of yourself <unk>


predicted:  <s> And <unk> <unk> , the the the the the the the <unk>
real:  <s> The third trait is <unk> , what we might call street smarts -- it <unk> a Greek word <unk>


predicted:  <s> And <unk> , , , , , , , , , , , , , <unk> , <unk>
real:  <s> When I find a chapter that I want to see , I just touch the text and the system will format pages for me to read <unk>


predicted:  <s> And <unk> <unk> <unk> <unk>
real:  <s> Let <unk> make sure you have it <unk>


predicted:  <s> And <unk> , <unk> , <unk> , <unk> <unk> <unk> <unk>
real:  <s> As you can see , I use my finger to interact with my digital body <unk>


predicted:  <s> And <unk> <unk> <unk> <u

predicted:  <s> And <unk> , , , , , , , , , , , the the the the the the the the the the <unk>
real:  <s> This <unk> cancer was growing throughout his left lung , and he had a chest tube in that was draining little bits of debris <unk>


predicted:  <s> And , the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
real:  <s> Daniel <unk> has linguistic , numerical and visual <unk> -- meaning that his perception of words , numbers and colors are woven together into a new way of perceiving and understanding the world . The author of <unk> Born on a Blue Day , <unk> <unk> shares his art


predicted:  <s> And <unk> , <unk> , <unk> , <unk> <unk> <unk> <unk>
real:  <s> You have to go to the literature and look up what <unk> known about feet <unk>


predicted:  <s> And , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , <unk>
real:  <s> We think ultimately all of th

KeyboardInterrupt: 